# Imports and functions

In [1]:
# Import Modules 
import arcpy
import time
import sys
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm, trange

In [2]:
# Costume Functions
#======= Function 1 ========#
# Returns the unique values of a list
# Inputs: list
def unique(list1):
    #import numpy as np
    x = np.array(list1)
    return list(np.unique(x))

#======= Function 2 ========#
# Converts fc table into dataframe
# Inputs: fc (feature class), fields (desired fields, list format)
# col_names (how you want to call the col on the dataframe, list
# format and same order as fields) - ALL REQUIERD
def fc_to_df(fc, fields, col_names):
    data = []
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        for row in cursor:
            dict_temp = {}
            
            for i in range(len(fields)):
                dict_temp.update({col_names[i]:row[i]})
                
            data.append(dict_temp)
            
    return(pd.DataFrame(data))

In [2]:
# Setting Inputs 
# set up inputs
input_fc = 'Digital_Acre_2022'
#input_fc = arcpy.GetParameterAsText(0)
plot = 'Rep'
#arcpy.GetParameterAsText(1)
row = 'Row_Numb'
#arcpy.GetParameterAsText(2)
zone = 'Zone_Numb'
#arcpy.GetParameterAsText(3)
plantn = 'Plant_NF'
#arcpy.GetParameterAsText(4)
coreID = 'Core_ID'
#arcpy.GetParameterAsText(5)

# Input fields transformed to list 
fields = [plot, row, zone, plantn, coreID]

# Skip Bowl Data Processing

## Adding fields - next to skip bowl and skip bowl property

In [12]:
# Adding 'SkipBowl' and 'SkipProp' fields to fc dataset
arcpy.management.AddField(input_fc, "SkipBowl", "SHORT",field_alias="Proximity_SkipBowl",
                          field_is_nullable="NULLABLE")
#arcpy.AddMessage('Adding SkipBowl field to ' + str(os.path.basename(input_fc)))

arcpy.management.AddField(input_fc, "SkipProp", "TEXT", field_length=10,
                          field_alias="SkipBowl_Property", field_is_nullable="NULLABLE")
#arcpy.AddMessage('Adding SkipProp field to ' + str(os.path.basename(input_fc)))

# Generating dataframe from input fc
plts_22 = fc_to_df(input_fc, fields, fields)

## Identidying plants next to skip bowl

In [35]:
# List for recording coreID of skps and their position 
skb_coreID = []
skb_pos = []

# Generating unique ist of reps and rows in order 
reps = unique(list(plts_22[fields[0]]))
reps.sort()
rows = unique(list(plts_22[fields[1]]))
rows.sort()

# Itiratin over every plant row of the dataset 
for rep in reps:
    for row in rows:
        # Subset of one field row 
        row_df = plts_22[(plts_22[fields[0]] == rep) & (plts_22[fields[1]] == row)]
        # How many zones does the row has?
        row_length = unique(row_df[fields[2]])
        
        # If statement that identify how many zones the row has
        # Dependig on the zone is how many plants are going to be flag 
        if len(row_length) == 2:
            #Last Plant of Zone 1
            zone_df = row_df[row_df[fields[2]] == min(row_length)].reset_index()
            plant_index = zone_df[fields[3]].idxmax()
            skb_coreID.append(zone_df.iloc[plant_index,5])
            skb_pos.append("UP")

            #First Plant of zone 2
            zone_df = row_df[row_df[fields[2]] == max(row_length)].reset_index()
            plant_index = zone_df[fields[3]].idxmin()
            skb_coreID.append(zone_df.iloc[plant_index,5])
            skb_pos.append("DOWN")


        if len(row_length) == 3:
            #Last Plant of Zone 1
            zone_df = row_df[row_df[fields[2]] == 1].reset_index()
            plant_index = zone_df[fields[3]].idxmax()
            skb_coreID.append(zone_df.iloc[plant_index,5])
            skb_pos.append("UP")

            #First Plant of zone 2
            zone_df = row_df[row_df[fields[2]] == 2].reset_index()
            plant_index = zone_df[fields[3]].idxmin()
            skb_coreID.append(zone_df.iloc[plant_index,5])
            skb_pos.append("DOWN")

            #Last Plant of Zone 2
            zone_df = row_df[row_df[fields[2]] == 2].reset_index()
            plant_index = zone_df[fields[3]].idxmax()
            skb_coreID.append(zone_df.iloc[plant_index,5])
            skb_pos.append("UP")

            #First Plant of zone 3
            zone_df = row_df[row_df[fields[2]] == 3].reset_index()
            plant_index = zone_df[fields[3]].idxmin()
            skb_coreID.append(zone_df.iloc[plant_index,5])
            skb_pos.append("DOWN")

zip_obj = zip(skb_coreID,skb_pos)
dict_skb = dict(zip_obj)

In [77]:
# Updating fc to containg UP/Down Property and skip_bowl = 1 
#arcpy.AddMessage('Uppdating ' + str(os.path.basename(input_fc)) + ' SkipBowl and SkipProp fields.')
fields.extend(['SkipBowl', 'SkipProp'])
#Updates input_fc 
with arcpy.da.UpdateCursor(input_fc, fields) as cursor:
    for row in cursor:
        if row[4] in skb_coreID:
            # Uses the Master/Core ID to identify wich plants have been tag and what is their property
            row[5] = 1                         
            row[6] = dict_skb[row[4]]
            cursor.updateRow(row)

## Ranking plants after skip bowl

In [39]:
original_project_workspace = arcpy.env.workspace 

In [107]:
# Calculating rank
arcpy.env.workspace = 'memory'

near_fid_list = []
near_rank_list = []

#Selecting only skip bowl plants 
where_clause = 'SkipBowl = 1'
skipb_plants = arcpy.MakeFeatureLayer_management(input_fc, os.path.join(arcpy.env.scratchWorkspace, 'skpb_plants'), where_clause)

#Generate near table for skip bowl plants 
arcpy.analysis.GenerateNearTable(skipb_plants, input_fc, 'test', angle = 'ANGLE', closest = 'ALL', closest_count = 20)

# Add azimuth angle field to near table 
arcpy.management.AddField('test', "AZIMUTH", "DOUBLE", field_alias="AZIMUTH", field_is_nullable="NULLABLE")

# Calculate azimuth angle and update table 
with arcpy.da.UpdateCursor('test', ["NEAR_ANGLE", "AZIMUTH"]) as cursor:
    for row in cursor:
        angle = row[0]
        if angle <= 180 and angle > 90:
            row[1] = (360.0 - (angle - 90))
        else:
            row[1] = (abs(angle - 90))
        cursor.updateRow(row)

# Recording all plants next to skip bowl OID
oid_values = [i[0] for i in arcpy.da.SearchCursor(skipb_plants, arcpy.Describe(skipb_plants).OIDFieldName)]

# Adding OID to skip bowl fields 
fields.append(arcpy.Describe(skipb_plants).OIDFieldName)

# Near table fields
near_fields = [nfield.name for nfield in arcpy.ListFields('test')]


with arcpy.da.SearchCursor(skipb_plants, fields) as cursor:
    # Itirates through skip bowl plants 
    for row in cursor:
        # Select all near cases for one of the skip bowl plants 
        where_clause = "{in_id} = {fc_oid}".format(in_id = near_fields[1], fc_oid = row[7])
        # Coursor for near table that only selects all cases of ONE skip plant 
        with arcpy.da.SearchCursor('test', near_fields, where_clause) as n_cursor:
            near_fid = []
            near_rank = []
            near_rank_res = []
            # Itirates through near table 
            for n_row in n_cursor:
                if row[6] == 'UP' and 250 < n_row[6] < 290:
                    near_fid.append(n_row[2])
                    near_rank.append(n_row[4])

                if row[6] == 'DOWN' and 70 < n_row[6] < 110:
                    near_fid.append(n_row[2])
                    near_rank.append(n_row[4])
                    

                near_rank_res = list(range(2,len(near_rank)+2))
        near_fid_list.extend(near_fid)
        near_rank_list.extend(near_rank_res)

In [108]:
near_zip = zip(near_fid_list, near_rank_list)
near_dict = dict(near_zip)

# Updating input fc to include rank 
with arcpy.da.UpdateCursor(input_fc, [arcpy.Describe(input_fc).OIDFieldName, 'SkipBowl']) as cursor:
    for row in cursor:
        if row[1] == None:
            try:
                row[1] = near_dict[row[0]]
                cursor.updateRow(row)
            except KeyError:
                pass

## Data cleaning of skip bowl proximity

In [115]:
with arcpy.da.UpdateCursor(input_fc, 'SkipBowl') as ucursor:
    for urow in ucursor:
        try:
            if (urow[0] > 7):
                urow[0] = None
                ucursor.updateRow(urow)
        except TypeError:
            pass

# Next and Previos Distance Calculations

In [10]:
prev_dist_dict = {}
next_dist_dict = {}

arcpy.env.workspace = r"C:\Users\abe_lizbethp\OneDrive - Iowa State University\Documents\2022_Data_Analysis\data_analysis_v2\data_analysis_v2.gdb"

# Generates near table for input_fc
neart = arcpy.analysis.GenerateNearTable(input_fc, input_fc, 'test', angle = 'ANGLE', closest = 'ALL', closest_count = 10)
# Adding azimth angle field 
arcpy.management.AddField(neart, "AZIMUTH", "DOUBLE", field_is_nullable="NULLABLE")
#Converting near angle to azimuth angle
in_table = neart
with arcpy.da.UpdateCursor(in_table, ["NEAR_ANGLE", "AZIMUTH"]) as cursor:
    for row in cursor:
        angle = row[0]
        if angle <= 180 and angle > 90:
            row[1] = (360.0 - (angle - 90))
            cursor.updateRow(row)
        else:
            row[1] = (abs(angle - 90))
            cursor.updateRow(row)


# List all OID values for input_fc
oid_values = [i[0] for i in arcpy.da.SearchCursor(input_fc, arcpy.Describe(input_fc).OIDFieldName)]

p_bar = tqdm(range(len(oid_values)))
for number in p_bar:
    oid = oid_values[number]
    # Obtaining NEXT distances for all objects in input_fc
    where_clause1 = "AZIMUTH >= 70 And AZIMUTH <= 110 And IN_FID = {fc_oid}".format(fc_oid = oid)
    all_dist_next = [i[0] for i in arcpy.da.SearchCursor(neart,'NEAR_DIST', where_clause1)]
    try:
        next_dist = min(all_dist_next)
        next_dist_dict[oid] = next_dist
    except ValueError:
        pass
    
    # Obtaining PREVIOUS distances for all objects in input_fc
    where_clause2= "AZIMUTH >= 250 And AZIMUTH <= 290 And IN_FID = {fc_oid}".format(fc_oid = oid)
    all_dist_prev = [i[0] for i in arcpy.da.SearchCursor(neart,'NEAR_DIST', where_clause2)]
    try:
        prev_dist = min(all_dist_prev)
        prev_dist_dict[oid] = prev_dist
    except ValueError:
        pass
    
    p_bar.set_description(f'Working on "{number}"')

  0%|          | 0/22557 [00:00<?, ?it/s]

In [13]:
arcpy.management.AddField(input_fc, "prevdist", "DOUBLE", field_is_nullable="NULLABLE")
arcpy.management.AddField(input_fc, "nextdist", "DOUBLE", field_is_nullable="NULLABLE")


with arcpy.da.UpdateCursor(input_fc, [arcpy.Describe(input_fc).OIDFieldName, 'prevdist', 'nextdist']) as cursor:
    for row in cursor:
        try:
            row[1] = prev_dist_dict[row[0]]
            row[2] = next_dist_dict[row[0]]
            cursor.updateRow(row)
        except KeyError:
            try:
                row[1] = prev_dist_dict[row[0]]
                cursor.updateRow(row)
            except KeyError:
                try:
                    row[2] = next_dist_dict[row[0]]
                    cursor.updateRow(row)
                except:
                    pass

# Determining 'Normal Spacing Plants'

In [4]:
with arcpy.da.UpdateCursor(input_fc, ['SkipBowl', 'prevdist', 'nextdist']) as cursor:
    for row in cursor:
        if row[0] == None:
            try:
                if (row[1] > 0.0762 and row[1] < 0.2286 and row[2] > 0.0762 and row[2] < 0.2286):
                    row[0] = -1
                    cursor.updateRow(row)
            except TypeError:
                pass
        else:
            pass